In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import datasets, linear_model,preprocessing
from datetime import datetime
import gc
%matplotlib inline
from IPython.display import display, HTML
from pprint import pprint
import time
import nltk.corpus
from nltk.stem.snowball import SnowballStemmer
from ml_metrics import rmse,rmsle

In [2]:
#take 1 CSV, then split it to 3..
class FeatureEngineering:

    def __init__(self, ValidationStart, ValidationEnd, trainHdfPath, trainHdfFile, testHdfPath1, testHdfPath2, testHdfFile, 
                 testTypes, trainTypes, trainCsvPath, testCsvPath, maxLag=0):
        self.ValidationStart = ValidationStart
        self.ValidationEnd = ValidationEnd
        self.maxLag = maxLag
        self.trainHdfPath = trainHdfPath
        self.trainHdfFile = trainHdfFile
        self.testHdfPath1 = testHdfPath1
        self.testHdfPath2 = testHdfPath2
        self.testHdfFile = testHdfFile
        self.testTypes = testTypes
        self.trainTypes = trainTypes
        self.trainCsvPath = trainCsvPath
        self.testCsvPath = testCsvPath
        
    @staticmethod
    def __printDataFrameBasics__(data):
        display(data.head(2))
        #print data.dtypes
        gc.collect()
        print(data.info(memory_usage=True))
        
    @staticmethod    
    def changeIndexTypeToLowerMemory(data):
        ##########
        #This is very critical, i accept max number is 2^32. Also, if don't do that, memory gets so much higher..
        ##########
        #data.reset_index(inplace=True)
        #data.drop("index",axis=1, inplace=True)
        #data.index = data.index.astype('uint32')
        gc.collect()
        
    def ReadHdf(self, trainOrTestOrBoth):
        '''Reads and holds Df in object memory'''            
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_hdf(self.trainHdfPath,self.trainHdfFile)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
            
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test1 = pd.read_hdf(self.testHdfPath1,self.testHdfFile)
            self.test2 = pd.read_hdf(self.testHdfPath2,self.testHdfFile)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test1)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test2)
            FeatureEngineering.__printDataFrameBasics__(self.test1)
            FeatureEngineering.__printDataFrameBasics__(self.test2)
        
    def ReadCsv(self, trainOrTestOrBoth):
        '''Reads and holds Df in memory'''
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth == 'both':
            self.train = pd.read_csv(self.trainCsvPath, usecols=self.trainTypes.keys(), dtype=self.trainTypes)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            tempTest = pd.read_csv(self.testCsvPath, usecols=self.testTypes.keys(), dtype=self.testTypes)
            self.test1 = tempTest.loc[tempTest.Semana.values == self.ValidationStart]
            self.test2 = tempTest.loc[tempTest.Semana.values == self.ValidationEnd]
            del tempTest
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test1)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test2)
            FeatureEngineering.__printDataFrameBasics__(self.test1)
            FeatureEngineering.__printDataFrameBasics__(self.test2)
            
    @staticmethod
    def ConvertCsvToHdf(csvPath, HdfPath, HdfName, ColumnTypeDict ):
        tempDf = pd.read_csv(csvPath, usecols=ColumnTypeDict.keys(), dtype=ColumnTypeDict,index=False)
        tempDf.to_hdf(HdfPath, HdfName, format='t')
        del tempDf
        gc.collect()
        print "ConvertCsvToHdf is done.."

    def Preprocess(self, trainOrTestOrBoth, columnFunctionTypeList):
        '''columnFunctionTypeList = [ ['C1',Func1,Type], ['C2',Func2,Type],..    ]'''
        for column, func, localType in columnFunctionTypeList:
            if trainOrTestOrBoth == 'train' or trainOrTestOrBoth =='both':
                self.train.loc[:,column] =  np.apply_along_axis(func,0,FE.train[column].values).astype(localType)
                #np.apply_along_axis(lambda x: x+1,0,FE.train["Semana"]).astype("int32")
            if trainOrTestOrBoth == 'test' or trainOrTestOrBoth == 'both':
                self.test1.loc[:,column] =  np.apply_along_axis(func,0,FE.test1[column].values).astype(localType)
                self.test2.loc[:,column] =  np.apply_along_axis(func,0,FE.test2[column].values).astype(localType)
        gc.collect()
        
    def SaveDataFrameToHdf(self,trainOrTestOrBoth):
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train.to_hdf(self.trainHdfPath, self.trainHdfFile, format='t', index="False")
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test1.to_hdf(self.testHdfPath1, self.testHdfFile, format='t', index="False")
            self.test2.to_hdf(self.testHdfPath2, self.testHdfFile, format='t', index="False")
        
    def AddDemandaGeneralMean(self,trainOrTestOrBoth): 
        #self.train.loc[:,"DemandaGeneralMean"] = self.train["Demanda_uni_equil"].loc[
         #   self.train.loc[:,'Semana'] < 10].mean().astype("float32")
            
        meanOfDemanda = self.train["Demanda_uni_equil"].values.mean().astype("float32")
        
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train.loc[:,"DemandaGeneralMean"] = meanOfDemanda
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test1.loc[:,"DemandaGeneralMean"] = meanOfDemanda
            self.test2.loc[:,"DemandaGeneralMean"] = meanOfDemanda
        
        #self.train.loc[:,"DemandaGeneralMean"] = self.train["Demanda_uni_equil"].values[
        #(self.train.loc[:,'Semana'].values < self.ValidationStart).values].mean().astype("float32")
        gc.collect()
        
    '''ConfigElements(0,[ ("A",["Semana","Agencia_ID"],["count","count"]),'''
    def AddConfigurableFeaturesToTrain(self, config):
        if config.lag > self.maxLag:
            self.maxLag = config.lag
        
        tempData = self.train[self.train['Semana'].values <= (self.ValidationEnd - config.lag)]
        #display(tempData)
        if(config.lag != 0):
            tempData.loc[:,'Semana'] = tempData['Semana'].values + config.lag
        #display(tempData)
        
        #Means iterative.. eliminate as long as np.nan exists..If there is already one, don't create, use the existing
        if config.targetVariable != "" and  config.targetVariable not in self.train.columns:
            self.train.loc[:,config.targetVariable] = np.nan
            self.test1.loc[:,config.targetVariable] = np.nan
            
            if config.lag != 1:
                self.test2.loc[:,config.targetVariable] = np.nan
        
        for name,groups,aggregate in config.nameAndGroups:
            if name not in self.train.columns:
                print "{} is not in columns..".format(name)            
                
                groupedDataframe = tempData[groups+['Demanda_uni_equil']].copy().groupby(groups).agg(aggregate[0])
                gc.collect()
                #groupedDataframe.columns = groupedDataframe.columns.droplevel(0)
                groupedDataframe.columns = [name]
                
                #This is means of the counts of the semana-columns tuples!..!!!
                #If no lag and mean, mean of the columns without semana!!..
                #If there is lag and count, count of the columns x weeks before
                #If there is lag and mean, mean of the columns x weeks before
                #if(config.lag == 0 and aggregate == "count"):
                if(len(aggregate)>1):
                    groupedDataframe.reset_index(inplace=True)
                    groupedDataframe.drop("Semana",axis=1, inplace=True)
                    groups = groups[1:]
                    groupedDataframe = groupedDataframe.groupby(groups).agg(aggregate[1])
                    groupedDataframe.columns = [name]
                    gc.collect()
                
                display(groupedDataframe.head(2))
                self.train = self.train.merge( groupedDataframe, left_on=groups,
                    right_index=True, how='left', sort=False,copy=False)
                gc.collect()
                self.test1 = self.test1.merge( groupedDataframe, left_on=groups,
                    right_index=True, how='left', sort=False,copy=False)
                gc.collect()
                if config.lag != 1:
                    self.test2 = self.test2.merge( groupedDataframe, left_on=groups,
                        right_index=True, how='left', sort=False,copy=False)
                
                del groupedDataframe
                gc.collect()
            else:
                print "{} is in columns..".format(name)
            
            display(self.train.head(2))
            display(self.test1.head(2))
            display(self.test2.head(2))
            
            #Means iterative..!!!!!
            if config.targetVariable != "":
                self.train.loc[pd.isnull(self.train[config.targetVariable].values), 
                    config.targetVariable] = self.train.loc[pd.isnull(self.train[config.targetVariable].values)
                    , name].values
                self.test1.loc[pd.isnull(self.test1[config.targetVariable].values), 
                    config.targetVariable] = self.test1.loc[pd.isnull(self.test1[config.targetVariable].values),
                    name].values
                if config.lag != 1:
                    self.test2.loc[pd.isnull(self.test2[config.targetVariable].values), 
                        config.targetVariable] = self.test2.loc[pd.isnull(self.test2[config.targetVariable].values)
                        , name].values
                    
                count = self.test1[config.targetVariable].isnull().sum()
                print "Count of missing numbers after {} in validation part 1 in column {} is {}".format(name, 
                    config.targetVariable,str(count))
                if config.lag != 1:
                    count = self.test2.loc[:,config.targetVariable].isnull().sum()
                    print "Count of missing numbers after {} in validation part 2 in column {} is {}".format(name, 
                        config.targetVariable,str(count))
                
                
                #display(self.train)
                #If column is already in Dataframe and we want to fill target variable, this deletes columns!!!
                if(config.deleteColumns):
                    self.train.drop(name, axis=1, inplace=True)
                    self.test1.drop(name, axis=1, inplace=True)
                    if config.lag != 1:
                        self.test2.drop(name, axis=1, inplace=True)
                gc.collect()
                #Only in tesst
                #if count == 0:
                 #   break
        del tempData
        display(self.train.head(2))
        display(self.test1.head(2))
        display(self.test2.head(2))
        gc.collect()
        return 
    
    def DeleteLaggedWeeksFromTrain(self):
        self.train = self.train[self.train['Semana'].values >= (3 + self.maxLag)]
        gc.collect()
        display(self.train.head(2))
        
    def ReadFirstNRowsOfACsv(self, nrows, trainOrTestOrBoth) :
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_csv(self.trainCsvPath, usecols=self.trainTypes.keys(), dtype=self.trainTypes, nrows = nrows)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            tempTest = pd.read_csv(self.testCsvPath, usecols=self.testTypes.keys(), dtype=self.testTypes, nrows = nrows*2)
            self.test1 = tempTest.loc[tempTest.Semana == self.ValidationStart]
            self.test2 = tempTest.loc[tempTest.Semana == self.ValidationEnd]
            del tempTest
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test1)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test2)
            FeatureEngineering.__printDataFrameBasics__(self.test1)
            FeatureEngineering.__printDataFrameBasics__(self.test2)
    
    #Use when concatanating train and validation before predict test for example..
    def AppendTestToTrain(self,deleteTest = True):
        self.train = self.train.append(self.test1,ignore_index=True)
        gc.collect()
        if(deleteTest):
            del self.test1
            gc.collect()
        try:
            self.train = self.train.append(self.test2,ignore_index=True)
            gc.collect()
            if(deleteTest):
                del self.test2
                gc.collect()
        except:
            pass
        #BAD PERFORMANCE!!
    #Split train data to train and test1 and test2 (validation)
    #def SplitTrainToTestUsingValidationStart(self):
     #   boolCondition = self.train.Semana == self.ValidationStart
      #  self.test1 = self.train.loc[boolCondition]
       # self.train.drop((self.train.loc[boolCondition].index), axis=0,inplace=True)
        
       # boolCondition = self.train.Semana == self.ValidationEnd
       # self.test2 = self.train.loc[boolCondition]
       # self.train.drop((self.train.loc[boolCondition].index), axis=0,inplace=True)
      #  del boolCondition
      #  gc.collect()
    
    #Reaches 3x memory from train, because of test1, test2 and train itself at the end.. GC fixed in the end..
    def SplitTrainToTestUsingValidationStart(self):
        boolCondition = self.train.Semana.values == self.ValidationStart
        self.test1 = self.train[boolCondition]
        boolCondition = self.train.Semana.values == self.ValidationEnd
        self.test2 = self.train[boolCondition]
        FE.train = FE.train[ FE.train.Semana.values < FE.ValidationStart ]
        del boolCondition
        gc.collect()

In [3]:
class ConfigElements:
    def __init__(self, lag, nameAndGroups, targetVariable="", deleteColumns = False):
        self.lag = lag
        self.nameAndGroups = nameAndGroups
        #If there is target variable, then 5 4 3 2 1, fill the np.nans..Else hold them all in Dataframe..
        self.targetVariable = targetVariable
        self.deleteColumns = deleteColumns

In [4]:
parameterDict =       {"ValidationStart":8, 
 "ValidationEnd":9,
   "maxLag":3,
    "trainHdfPath":'../../input/train.h5',
    "trainHdfFile":"train",
    "testHdfPath1":"../../input/test1.h5",
    "testHdfPath2":"../../input/test2.h5",
    "testHdfFile":"test", 
    "trainTypes" : {'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16, 
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16,'Venta_uni_hoy':np.uint16, 'Venta_hoy':np.float32,
                    'Dev_uni_proxima': np.uint32, 'Dev_proxima':np.float32,'Demanda_uni_equil':np.uint32}, 
    "testTypes" : {'id':np.uint32,'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16,
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16},
    "trainCsvPath":'../../input/train.csv'   ,
    "testCsvPath":'../../input/test.csv'}

FE = FeatureEngineering(**parameterDict)
print FE.__dict__

{'trainCsvPath': '../../input/train.csv', 'maxLag': 3, 'testTypes': {'Cliente_ID': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Semana': <type 'numpy.uint8'>, 'id': <type 'numpy.uint32'>}, 'testHdfFile': 'test', 'trainTypes': {'Dev_proxima': <type 'numpy.float32'>, 'Venta_uni_hoy': <type 'numpy.uint16'>, 'Cliente_ID': <type 'numpy.uint32'>, 'Demanda_uni_equil': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Venta_hoy': <type 'numpy.float32'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Dev_uni_proxima': <type 'numpy.uint32'>, 'Semana': <type 'numpy.uint8'>}, 'testHdfPath1': '../../input/test1.h5', 'ValidationEnd': 9, 'testHdfPath2': '../../input/test2.h5', 'testCsvPath': '../../input/test.csv', 'ValidationStart': 8, 'trainHdfFile': 'train', 'trainHdfPath': '../../input/train.h5'}


In [112]:
FE.ReadCsv('train')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,3
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74180464 entries, 0 to 74180463
Data columns (total 11 columns):
Semana               uint8
Agencia_ID           uint16
Canal_ID             uint8
Ruta_SAK             uint16
Cliente_ID           uint32
Producto_ID          uint16
Venta_uni_hoy        uint16
Venta_hoy            float32
Dev_uni_proxima      uint32
Dev_proxima          float32
Demanda_uni_equil    uint32
dtypes: float32(2), uint16(4), uint32(3), uint8(2)
memory usage: 2.1 GB
None


## Merge town and products..

In [113]:
townstate = pd.read_csv("../../input/town_state.csv", encoding='utf-8')
townstate['Town_ID']=townstate['Town'].str[:4]
states = townstate['State']
le = preprocessing.LabelEncoder()
townstate['State_ID']=le.fit_transform(states)
townstate = townstate.drop(['Town', 'State'], axis=1)
townstate = townstate.astype('uint16')
townstate[['State_ID']] =townstate[['State_ID']] .astype('uint8')
FE.train = pd.merge(FE.train, townstate, on='Agencia_ID', how='left', sort=False,copy=False)
gc.collect()

211

In [114]:
products = pd.read_csv("../../input/producto_tabla.csv")
products['short_name'] = products.NombreProducto.str.extract('^(\D*)', expand=False)
products['brand'] = products.NombreProducto.str.extract('^.+\s(\D+) \d+$', expand=False)
w = products.NombreProducto.str.extract('(\d+)(Kg|g)', expand=True)
products['weight'] = w[0].astype('float') * w[1].map({'Kg': 1000, 'g': 1})
products['pieces'] = products.NombreProducto.str.extract('(\d+)p ', expand=False).astype('float')

products['short_name_processed'] = (products['short_name'].
                                    map(lambda x: " ".
                                        join([i for i in x.lower().split() 
                                              if i not in nltk.corpus.stopwords.words("spanish")])))
stemmer = SnowballStemmer("spanish")
products['short_name_processed'] = (products['short_name_processed'].
                                    map(lambda x: " ".join([stemmer.stem(i) for i in x.lower().split()])))

le = preprocessing.LabelEncoder()

products['Prod_name_ID']=le.fit_transform(products['short_name_processed'])
products['Brand_ID']=le.fit_transform(products['brand'])

products = products.drop(['short_name', 'brand', 'short_name_processed', 'NombreProducto'], axis=1)
products.fillna(value=0, inplace=True)
products[['pieces','Brand_ID']] = products[['pieces','Brand_ID']].astype('uint8')
products[['Producto_ID','weight', 'Prod_name_ID']] = products[['Producto_ID','weight', 'Prod_name_ID']].astype('uint16')
FE.train = pd.merge(FE.train, products, on='Producto_ID', how='left', sort=False,copy=False)
gc.collect()

104

## Check NA values here.

In [115]:
#FE.train[FE.train.isnull().any(axis=1)]


## Convert to Log..

In [116]:
FE.train.loc[:,"Demanda_uni_equil"] = np.log1p (FE.train["Demanda_uni_equil"].values)
FE.train.loc[:,"Demanda_uni_equil_original"] =  np.round( np.expm1(FE.train["Demanda_uni_equil"]))
gc.collect()

7

## Split Train to test1 test2

In [117]:
FE.SplitTrainToTestUsingValidationStart()

## Add DemandaNotEqualTheDifferenceOfVentaUniAndDev to model products delivered later..

In [118]:
FE.train.loc[:,"DemandaNotEqualTheDifferenceOfVentaUniAndDev"] = FE.train.Demanda_uni_equil_original.values < (
    FE.train.Venta_uni_hoy.values - FE.train.Dev_uni_proxima.values)
gc.collect()

0

## Delete Demanda = 0  and Venta = 0

In [119]:
FE.train = FE.train.loc[~((FE.train.Demanda_uni_equil_original == 0) & (FE.train.Venta_uni_hoy == 0))]
gc.collect()

33

In [120]:
FE.SaveDataFrameToHdf('both')

In [5]:
FE.ReadHdf('both')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0,Lag1,Lag2
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,16,120,2,709,4,3.0,False,1.595831,0.0,0.0
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,16,135,2,712,4,4.0,False,1.177221,0.0,0.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 53165116 entries, 0 to 53364882
Data columns (total 22 columns):
Semana                                          uint8
Agencia_ID                                      uint16
Canal_ID                                        uint8
Ruta_SAK                                        uint16
Cliente_ID                                      uint32
Producto_ID                                     uint16
Venta_uni_hoy                                   uint16
Venta_hoy                                       float32
Dev_uni_proxima                                 uint32
Dev_proxima                                     float32
Demanda_uni_equil                               float64
Town_ID                                         uint16
State_ID                                        uint8
weight                                          uint16
pieces                                          uint8
Prod_name_ID                                    uint16
Brand_

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,Lag0,Lag1,Lag2
53364883,8,1110,7,3301,15766,1212,4,33.520000,0,0.0,...,2008,16,120,2,709,4,4.0,1.595831,1.499550,1.450554
53364884,8,1110,7,3301,15766,1216,5,41.900002,0,0.0,...,2008,16,135,2,712,4,5.0,1.177221,1.098612,0.693147


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10406868 entries, 53364883 to 63771750
Data columns (total 21 columns):
Semana                        uint8
Agencia_ID                    uint16
Canal_ID                      uint8
Ruta_SAK                      uint16
Cliente_ID                    uint32
Producto_ID                   uint16
Venta_uni_hoy                 uint16
Venta_hoy                     float32
Dev_uni_proxima               uint32
Dev_proxima                   float32
Demanda_uni_equil             float64
Town_ID                       uint16
State_ID                      uint8
weight                        uint16
pieces                        uint8
Prod_name_ID                  uint16
Brand_ID                      uint8
Demanda_uni_equil_original    float64
Lag0                          float64
Lag1                          float64
Lag2                          float64
dtypes: float32(2), float64(5), uint16(7), uint32(2), uint8(5)
memory usage: 823.8 MB
None


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,Lag0,Lag2
63771751,9,1110,7,3301,15766,1212,1,8.38,0,0.0,0.693147,2008,16,120,2,709,4,1.0,1.595831,1.499550
63771752,9,1110,7,3301,15766,1238,2,19.66,0,0.0,1.098612,2008,16,140,2,630,4,2.0,1.177221,1.098612


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10408713 entries, 63771751 to 74180463
Data columns (total 20 columns):
Semana                        uint8
Agencia_ID                    uint16
Canal_ID                      uint8
Ruta_SAK                      uint16
Cliente_ID                    uint32
Producto_ID                   uint16
Venta_uni_hoy                 uint16
Venta_hoy                     float32
Dev_uni_proxima               uint32
Dev_proxima                   float32
Demanda_uni_equil             float64
Town_ID                       uint16
State_ID                      uint8
weight                        uint16
pieces                        uint8
Prod_name_ID                  uint16
Brand_ID                      uint8
Demanda_uni_equil_original    float64
Lag0                          float64
Lag2                          float64
dtypes: float32(2), float64(4), uint16(7), uint32(2), uint8(5)
memory usage: 744.5 MB
None


## Grouping for Lag0

In [122]:
#FE.train = FE.train.drop(['Lag0'], axis=1)  ## run these again, then comment out.

In [123]:
configLag0Target1DeleteColumnsFalse = ConfigElements(0,[ ("SPClRACh0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClRA0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SPClRCh0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClACh0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClR0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SPClA0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPClCh0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPCl0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SPR0_mean",[
                                                              "Producto_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SPA0_mean",[
                                                              "Producto_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPCh0_mean",[
                                                              "Producto_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPT0_mean",[
                                                              "Producto_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SPSt0_mean",[
                                                              "Producto_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SP0_mean",[
                                                              "Producto_ID"],
                                                          ["mean"]),
                                                      ("SPnClRACh0_mean",[  ##PRODNAMES START
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClRA0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SPnClRCh0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClACh0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClR0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SPnClA0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPnClCh0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPnCl0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SPnR0_mean",[
                                                              "Prod_name_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SPnA0_mean",[
                                                              "Prod_name_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPnCh0_mean",[
                                                              "Prod_name_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPnT0_mean",[
                                                              "Prod_name_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SPnSt0_mean",[
                                                              "Prod_name_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SPn0_mean",[
                                                              "Prod_name_ID"],
                                                          ["mean"]),
                                                       ("SBClRACh0_mean",[ ##brand id START
                                                              "Brand_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SBClRA0_mean",[
                                                              "Brand_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SBClRCh0_mean",[
                                                              "Brand_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SBClACh0_mean",[
                                                              "Brand_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SBClR0_mean",[
                                                              "Brand_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SBClA0_mean",[
                                                              "Brand_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SBClCh0_mean",[
                                                              "Brand_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SBCl0_mean",[
                                                              "Brand_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SBR0_mean",[
                                                              "Brand_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SBA0_mean",[
                                                              "Brand_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SBCh0_mean",[
                                                              "Brand_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SBT0_mean",[
                                                              "Brand_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SBSt0_mean",[
                                                              "Brand_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SB0_mean",[
                                                              "Brand_ID"],
                                                          ["mean"])
                                                       
                                                       
                                                       ], "Lag0", True)
print  configLag0Target1DeleteColumnsFalse.__dict__

{'nameAndGroups': [('SPClRACh0_mean', ['Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Agencia_ID', 'Canal_ID'], ['mean']), ('SPClRA0_mean', ['Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Agencia_ID'], ['mean']), ('SPClRCh0_mean', ['Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Canal_ID'], ['mean']), ('SPClACh0_mean', ['Producto_ID', 'Cliente_ID', 'Agencia_ID', 'Canal_ID'], ['mean']), ('SPClR0_mean', ['Producto_ID', 'Cliente_ID', 'Ruta_SAK'], ['mean']), ('SPClA0_mean', ['Producto_ID', 'Cliente_ID', 'Agencia_ID'], ['mean']), ('SPClCh0_mean', ['Producto_ID', 'Cliente_ID', 'Canal_ID'], ['mean']), ('SPCl0_mean', ['Producto_ID', 'Cliente_ID'], ['mean']), ('SPR0_mean', ['Producto_ID', 'Ruta_SAK'], ['mean']), ('SPA0_mean', ['Producto_ID', 'Agencia_ID'], ['mean']), ('SPCh0_mean', ['Producto_ID', 'Canal_ID'], ['mean']), ('SPT0_mean', ['Producto_ID', 'Town_ID'], ['mean']), ('SPSt0_mean', ['Producto_ID', 'State_ID'], ['mean']), ('SP0_mean', ['Producto_ID'], ['mean']), ('SPnClRACh0_mean', ['Prod_name_ID', 'Clien

In [124]:
FE.AddConfigurableFeaturesToTrain(configLag0Target1DeleteColumnsFalse)

SPClRACh0_mean is not in columns..


SPClRACh0_mean
Producto_ID Cliente_ID Ruta_SAK Agencia_ID Canal_ID                
41          146030     3303     2281       7               4.186777
            681747     3306     2281       7               7.364178

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0,SPClRACh0_mean
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,2008,16,120,2,709,4,3.0,False,NaN,1.595831
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,2008,16,135,2,712,4,4.0,False,NaN,1.177221


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,Lag0,SPClRACh0_mean
53364883,8,1110,7,3301,15766,1212,4,33.520000,0,0.0,1.609438,2008,16,120,2,709,4,4.0,NaN,1.595831
53364884,8,1110,7,3301,15766,1216,5,41.900002,0,0.0,1.791759,2008,16,135,2,712,4,5.0,NaN,1.177221


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,Lag0,SPClRACh0_mean
63771751,9,1110,7,3301,15766,1212,1,8.38,0,0.0,0.693147,2008,16,120,2,709,4,1.0,NaN,1.595831
63771752,9,1110,7,3301,15766,1238,2,19.66,0,0.0,1.098612,2008,16,140,2,630,4,2.0,NaN,1.177221


...

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,Lag0,SB0_mean
63771751,9,1110,7,3301,15766,1212,1,8.38,0,0.0,0.693147,2008,16,120,2,709,4,1.0,1.595831,1.573704
63771752,9,1110,7,3301,15766,1238,2,19.66,0,0.0,1.098612,2008,16,140,2,630,4,2.0,1.177221,1.573704


Count of missing numbers after SB0_mean in validation part 1 in column Lag0 is 0
Count of missing numbers after SB0_mean in validation part 2 in column Lag0 is 1


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,1.386294,2008,16,120,2,709,4,3.0,False,1.595831
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,1.609438,2008,16,135,2,712,4,4.0,False,1.177221


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,Lag0
53364883,8,1110,7,3301,15766,1212,4,33.520000,0,0.0,1.609438,2008,16,120,2,709,4,4.0,1.595831
53364884,8,1110,7,3301,15766,1216,5,41.900002,0,0.0,1.791759,2008,16,135,2,712,4,5.0,1.177221


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,Lag0
63771751,9,1110,7,3301,15766,1212,1,8.38,0,0.0,0.693147,2008,16,120,2,709,4,1.0,1.595831
63771752,9,1110,7,3301,15766,1238,2,19.66,0,0.0,1.098612,2008,16,140,2,630,4,2.0,1.177221


In [125]:
FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)

In [126]:
FE.SaveDataFrameToHdf('both')

In [127]:
print ('RMSLE Score:', rmse(FE.train.Demanda_uni_equil, FE.train.Lag0))
print ('RMSLE Score:', rmse(FE.test1.Demanda_uni_equil, FE.test1.Lag0))
print ('RMSLE Score:', rmse(FE.test2.Demanda_uni_equil, FE.test2.Lag0))

('RMSLE Score:', 0.30038675186633529)
('RMSLE Score:', 0.50754653787245785)
('RMSLE Score:', 0.52010612895052211)


In [128]:
configLag0Target1DeleteColumnsFalse = ConfigElements(1,[ ("SPClRACh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClRA0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SPClRCh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClACh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClR0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SPClA0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPClCh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPCl0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SPR0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SPA0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPCh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPT0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SPSt0_mean",["Semana",
                                                              "Producto_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SP0_mean",["Semana",
                                                              "Producto_ID"],
                                                          ["mean"]),
                                                      ("SPnClRACh0_mean",["Semana",  ##PRODNAMES START
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClRA0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SPnClRCh0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClACh0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClR0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SPnClA0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPnClCh0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPnCl0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SPnR0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SPnA0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPnCh0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPnT0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SPnSt0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SPn0_mean",["Semana",
                                                              "Prod_name_ID"],
                                                          ["mean"])

                                                       
                                                       ], "Lag1", True)
print  configLag0Target1DeleteColumnsFalse.__dict__

{'nameAndGroups': [('SPClRACh0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Agencia_ID', 'Canal_ID'], ['mean']), ('SPClRA0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Agencia_ID'], ['mean']), ('SPClRCh0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Canal_ID'], ['mean']), ('SPClACh0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Agencia_ID', 'Canal_ID'], ['mean']), ('SPClR0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Ruta_SAK'], ['mean']), ('SPClA0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Agencia_ID'], ['mean']), ('SPClCh0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Canal_ID'], ['mean']), ('SPCl0_mean', ['Semana', 'Producto_ID', 'Cliente_ID'], ['mean']), ('SPR0_mean', ['Semana', 'Producto_ID', 'Ruta_SAK'], ['mean']), ('SPA0_mean', ['Semana', 'Producto_ID', 'Agencia_ID'], ['mean']), ('SPCh0_mean', ['Semana', 'Producto_ID', 'Canal_ID'], ['mean']), ('SPT0_mean', ['Semana', 'Producto_ID', 'Town_ID'], ['mean']), ('SPSt0_mean', ['Se

In [129]:
FE.AddConfigurableFeaturesToTrain(configLag0Target1DeleteColumnsFalse)

SPClRACh0_mean is not in columns..


SPClRACh0_mean
Semana Producto_ID Cliente_ID Ruta_SAK Agencia_ID Canal_ID                
4      41          681747     3306     2281       7               7.632886
                   684023     3303     2281       7               3.433987

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0,Lag1,SPClRACh0_mean
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,16,120,2,709,4,3.0,False,1.595831,NaN,NaN
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,16,135,2,712,4,4.0,False,1.177221,NaN,NaN


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,Lag0,Lag1,SPClRACh0_mean
53364883,8,1110,7,3301,15766,1212,4,33.520000,0,0.0,...,2008,16,120,2,709,4,4.0,1.595831,NaN,NaN
53364884,8,1110,7,3301,15766,1216,5,41.900002,0,0.0,...,2008,16,135,2,712,4,5.0,1.177221,NaN,1.098612


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,Lag0
63771751,9,1110,7,3301,15766,1212,1,8.38,0,0.0,0.693147,2008,16,120,2,709,4,1.0,1.595831
63771752,9,1110,7,3301,15766,1238,2,19.66,0,0.0,1.098612,2008,16,140,2,630,4,2.0,1.177221


...

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,Lag0
63771751,9,1110,7,3301,15766,1212,1,8.38,0,0.0,0.693147,2008,16,120,2,709,4,1.0,1.595831
63771752,9,1110,7,3301,15766,1238,2,19.66,0,0.0,1.098612,2008,16,140,2,630,4,2.0,1.177221


Count of missing numbers after SPn0_mean in validation part 1 in column Lag1 is 46


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0,Lag1
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,2008,16,120,2,709,4,3.0,False,1.595831,NaN
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,2008,16,135,2,712,4,4.0,False,1.177221,NaN


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,Lag0,Lag1
53364883,8,1110,7,3301,15766,1212,4,33.520000,0,0.0,1.609438,2008,16,120,2,709,4,4.0,1.595831,1.499550
53364884,8,1110,7,3301,15766,1216,5,41.900002,0,0.0,1.791759,2008,16,135,2,712,4,5.0,1.177221,1.098612


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,Lag0
63771751,9,1110,7,3301,15766,1212,1,8.38,0,0.0,0.693147,2008,16,120,2,709,4,1.0,1.595831
63771752,9,1110,7,3301,15766,1238,2,19.66,0,0.0,1.098612,2008,16,140,2,630,4,2.0,1.177221


In [130]:
FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)

In [131]:
FE.SaveDataFrameToHdf('both')

In [132]:
configLag0Target1DeleteColumnsFalse = ConfigElements(2,[ ("SPClRACh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClRA0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SPClRCh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClACh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClR0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SPClA0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPClCh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPCl0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SPR0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SPA0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPCh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPT0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SPSt0_mean",["Semana",
                                                              "Producto_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SP0_mean",["Semana",
                                                              "Producto_ID"],
                                                          ["mean"]),
                                                      ("SPnClRACh0_mean",["Semana",  ##PRODNAMES START
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClRA0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SPnClRCh0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClACh0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClR0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SPnClA0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPnClCh0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPnCl0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SPnR0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SPnA0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPnCh0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPnT0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SPnSt0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SPn0_mean",["Semana",
                                                              "Prod_name_ID"],
                                                          ["mean"])
                                                       
                                                       ], "Lag2", True)
print  configLag0Target1DeleteColumnsFalse.__dict__

{'nameAndGroups': [('SPClRACh0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Agencia_ID', 'Canal_ID'], ['mean']), ('SPClRA0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Agencia_ID'], ['mean']), ('SPClRCh0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Canal_ID'], ['mean']), ('SPClACh0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Agencia_ID', 'Canal_ID'], ['mean']), ('SPClR0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Ruta_SAK'], ['mean']), ('SPClA0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Agencia_ID'], ['mean']), ('SPClCh0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Canal_ID'], ['mean']), ('SPCl0_mean', ['Semana', 'Producto_ID', 'Cliente_ID'], ['mean']), ('SPR0_mean', ['Semana', 'Producto_ID', 'Ruta_SAK'], ['mean']), ('SPA0_mean', ['Semana', 'Producto_ID', 'Agencia_ID'], ['mean']), ('SPCh0_mean', ['Semana', 'Producto_ID', 'Canal_ID'], ['mean']), ('SPT0_mean', ['Semana', 'Producto_ID', 'Town_ID'], ['mean']), ('SPSt0_mean', ['Se

In [133]:
FE.AddConfigurableFeaturesToTrain(configLag0Target1DeleteColumnsFalse)

SPClRACh0_mean is not in columns..


SPClRACh0_mean
Semana Producto_ID Cliente_ID Ruta_SAK Agencia_ID Canal_ID                
5      41          681747     3306     2281       7               7.632886
                   684023     3303     2281       7               3.433987

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0,Lag1,Lag2,SPClRACh0_mean
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,120,2,709,4,3.0,False,1.595831,0.0,NaN,NaN
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,135,2,712,4,4.0,False,1.177221,0.0,NaN,NaN


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,Lag0,Lag1,Lag2,SPClRACh0_mean
53364883,8,1110,7,3301,15766,1212,4,33.520000,0,0.0,...,16,120,2,709,4,4.0,1.595831,1.499550,NaN,NaN
53364884,8,1110,7,3301,15766,1216,5,41.900002,0,0.0,...,16,135,2,712,4,5.0,1.177221,1.098612,NaN,0.693147


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,Lag0,Lag2,SPClRACh0_mean
63771751,9,1110,7,3301,15766,1212,1,8.38,0,0.0,...,2008,16,120,2,709,4,1.0,1.595831,NaN,NaN
63771752,9,1110,7,3301,15766,1238,2,19.66,0,0.0,...,2008,16,140,2,630,4,2.0,1.177221,NaN,1.098612


...

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,Lag0,Lag2,SPn0_mean
63771751,9,1110,7,3301,15766,1212,1,8.38,0,0.0,...,2008,16,120,2,709,4,1.0,1.595831,1.499550,1.140908
63771752,9,1110,7,3301,15766,1238,2,19.66,0,0.0,...,2008,16,140,2,630,4,2.0,1.177221,1.098612,1.219895


Count of missing numbers after SPn0_mean in validation part 1 in column Lag2 is 17631
Count of missing numbers after SPn0_mean in validation part 2 in column Lag2 is 4110


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0,Lag1,Lag2
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,16,120,2,709,4,3.0,False,1.595831,0.0,NaN
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,16,135,2,712,4,4.0,False,1.177221,0.0,NaN


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,Lag0,Lag1,Lag2
53364883,8,1110,7,3301,15766,1212,4,33.520000,0,0.0,...,2008,16,120,2,709,4,4.0,1.595831,1.499550,1.450554
53364884,8,1110,7,3301,15766,1216,5,41.900002,0,0.0,...,2008,16,135,2,712,4,5.0,1.177221,1.098612,0.693147


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,Lag0,Lag2
63771751,9,1110,7,3301,15766,1212,1,8.38,0,0.0,0.693147,2008,16,120,2,709,4,1.0,1.595831,1.499550
63771752,9,1110,7,3301,15766,1238,2,19.66,0,0.0,1.098612,2008,16,140,2,630,4,2.0,1.177221,1.098612


In [134]:
FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)

In [135]:
FE.SaveDataFrameToHdf('both')

In [6]:
configLag0Target1DeleteColumnsFalse = ConfigElements(3,[ ("SPClRACh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClRA0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SPClRCh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClACh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClR0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SPClA0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPClCh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPCl0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SPR0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SPA0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPCh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPT0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SPSt0_mean",["Semana",
                                                              "Producto_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SP0_mean",["Semana",
                                                              "Producto_ID"],
                                                          ["mean"]),
                                                      ("SPnClRACh0_mean",["Semana",  ##PRODNAMES START
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClRA0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SPnClRCh0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClACh0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClR0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SPnClA0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPnClCh0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPnCl0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SPnR0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SPnA0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPnCh0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPnT0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SPnSt0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SPn0_mean",["Semana",
                                                              "Prod_name_ID"],
                                                          ["mean"])
                                                       
                                                       ], "Lag3", True)
print  configLag0Target1DeleteColumnsFalse.__dict__

{'nameAndGroups': [('SPClRACh0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Agencia_ID', 'Canal_ID'], ['mean']), ('SPClRA0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Agencia_ID'], ['mean']), ('SPClRCh0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Canal_ID'], ['mean']), ('SPClACh0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Agencia_ID', 'Canal_ID'], ['mean']), ('SPClR0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Ruta_SAK'], ['mean']), ('SPClA0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Agencia_ID'], ['mean']), ('SPClCh0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Canal_ID'], ['mean']), ('SPCl0_mean', ['Semana', 'Producto_ID', 'Cliente_ID'], ['mean']), ('SPR0_mean', ['Semana', 'Producto_ID', 'Ruta_SAK'], ['mean']), ('SPA0_mean', ['Semana', 'Producto_ID', 'Agencia_ID'], ['mean']), ('SPCh0_mean', ['Semana', 'Producto_ID', 'Canal_ID'], ['mean']), ('SPT0_mean', ['Semana', 'Producto_ID', 'Town_ID'], ['mean']), ('SPSt0_mean', ['Se

In [7]:
FE.AddConfigurableFeaturesToTrain(configLag0Target1DeleteColumnsFalse)

/home/cankoklu/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


SPClRACh0_mean is not in columns..


SPClRACh0_mean
Semana Producto_ID Cliente_ID Ruta_SAK Agencia_ID Canal_ID                
6      41          681747     3306     2281       7               7.632886
                   684023     3303     2281       7               3.433987

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0,Lag1,Lag2,Lag3,SPClRACh0_mean
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,2,709,4,3.0,False,1.595831,0.0,0.0,NaN,NaN
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,2,712,4,4.0,False,1.177221,0.0,0.0,NaN,NaN


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,Lag0,Lag1,Lag2,Lag3,SPClRACh0_mean
53364883,8,1110,7,3301,15766,1212,4,33.520000,0,0.0,...,120,2,709,4,4.0,1.595831,1.499550,1.450554,NaN,1.791759
53364884,8,1110,7,3301,15766,1216,5,41.900002,0,0.0,...,135,2,712,4,5.0,1.177221,1.098612,0.693147,NaN,1.386294


...

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,Lag0,Lag2,Lag3,SPn0_mean
63771751,9,1110,7,3301,15766,1212,1,8.38,0,0.0,...,16,120,2,709,4,1.0,1.595831,1.499550,1.450554,1.147519
63771752,9,1110,7,3301,15766,1238,2,19.66,0,0.0,...,16,140,2,630,4,2.0,1.177221,1.098612,1.098612,1.229737


Count of missing numbers after SPn0_mean in validation part 1 in column Lag3 is 18155
Count of missing numbers after SPn0_mean in validation part 2 in column Lag3 is 28484


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0,Lag1,Lag2,Lag3
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,120,2,709,4,3.0,False,1.595831,0.0,0.0,NaN
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,135,2,712,4,4.0,False,1.177221,0.0,0.0,NaN


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,Lag0,Lag1,Lag2,Lag3
53364883,8,1110,7,3301,15766,1212,4,33.520000,0,0.0,...,16,120,2,709,4,4.0,1.595831,1.499550,1.450554,1.791759
53364884,8,1110,7,3301,15766,1216,5,41.900002,0,0.0,...,16,135,2,712,4,5.0,1.177221,1.098612,0.693147,1.386294


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,Lag0,Lag2,Lag3
63771751,9,1110,7,3301,15766,1212,1,8.38,0,0.0,...,2008,16,120,2,709,4,1.0,1.595831,1.499550,1.450554
63771752,9,1110,7,3301,15766,1238,2,19.66,0,0.0,...,2008,16,140,2,630,4,2.0,1.177221,1.098612,1.098612


In [8]:
FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)

In [9]:
FE.SaveDataFrameToHdf('both')

In [10]:
print ('RMSLE Score:', rmse(FE.train.Demanda_uni_equil, FE.train.Lag0))
print ('RMSLE Score:', rmse(FE.test1.Demanda_uni_equil, FE.test1.Lag0))
print ('RMSLE Score:', rmse(FE.test2.Demanda_uni_equil, FE.test2.Lag0))


print ('RMSLE Score:', rmse(FE.train.Demanda_uni_equil, FE.train.Lag1))
print ('RMSLE Score:', rmse(FE.test1.Demanda_uni_equil, FE.test1.Lag1))
#print ('RMSLE Score:', rmse(FE.test2.Demanda_uni_equil, FE.test2.Lag1))


print ('RMSLE Score:', rmse(FE.train.Demanda_uni_equil, FE.train.Lag2))
print ('RMSLE Score:', rmse(FE.test1.Demanda_uni_equil, FE.test1.Lag2))
print ('RMSLE Score:', rmse(FE.test2.Demanda_uni_equil, FE.test2.Lag2))


print ('RMSLE Score:', rmse(FE.train.Demanda_uni_equil, FE.train.Lag3))
print ('RMSLE Score:', rmse(FE.test1.Demanda_uni_equil, FE.test1.Lag3))
print ('RMSLE Score:', rmse(FE.test2.Demanda_uni_equil, FE.test2.Lag3))

('RMSLE Score:', 0.30038675186633529)
('RMSLE Score:', 0.50754653787245785)
('RMSLE Score:', 0.52010612895052211)
('RMSLE Score:', 0.9575445053213959)
('RMSLE Score:', 0.56959833299659579)
('RMSLE Score:', 1.2403672595306536)
('RMSLE Score:', 0.58765631145354802)
('RMSLE Score:', 0.57931740650487062)
('RMSLE Score:', 1.461499636175799)
('RMSLE Score:', 0.58909846386851539)
('RMSLE Score:', 0.59567966992756638)


In [11]:
print ('RMSLE Score:', rmse(FE.train.Demanda_uni_equil, FE.train.Lag0))



print ('RMSLE Score:', rmse(FE.train.Demanda_uni_equil, FE.train.Lag1))



print ('RMSLE Score:', rmse(FE.train.Demanda_uni_equil, FE.train.Lag2))



print ('RMSLE Score:', rmse(FE.train.Demanda_uni_equil, FE.train.Lag3))


('RMSLE Score:', 0.30038675186633529)
('RMSLE Score:', 0.9575445053213959)
('RMSLE Score:', 1.2403672595306536)
('RMSLE Score:', 1.461499636175799)


In [12]:
FE.train.loc[:,"weightppieces"] = (FE.train["weight"].values/FE.train["pieces"].values)
FE.test1.loc[:,"weightppieces"] = (FE.test1["weight"].values/FE.test1["pieces"].values)
FE.test2.loc[:,"weightppieces"] = (FE.test2["weight"].values/FE.test2["pieces"].values)
FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)
gc.collect()
FE.SaveDataFrameToHdf('both')

clients per town
clietn sum venta hoy client sum venta uni hoy -
prod  id sum venta hoy prod id sum venta uni hoy -
prod name id sum venta hoy prod name id  sum venta uni hoy -
product name id sum demanda - 

grouping = ['Semana', 'Cliente_ID', 'Producto_ID']
train_group = train.groupby(grouping,as_index=False)['Demanda_uni_equil'].sum()

In [13]:
client_sum_venta_hoy = FE.train.groupby('Cliente_ID',as_index=False)['Venta_hoy'].sum()
client_sum_venta_uni_hoy = FE.train.groupby('Cliente_ID',as_index=False)['Venta_uni_hoy'].sum()
client_sum = pd.merge(client_sum_venta_hoy, client_sum_venta_uni_hoy, on='Cliente_ID', how='left', sort=False,copy=False)
client_sum.loc[:,"client_sum_venta_div_venta_uni"] = (client_sum["Venta_hoy"].values/client_sum["Venta_uni_hoy"].values)
client_sum.columns = ['Cliente_ID', 'Client_Sum_Venta_hoy', 'Client_Sum_Venta_uni_hoy', 'Client_Sum_venta_div_venta_uni']

In [14]:
client_sum.head()

,Cliente_ID,Client_Sum_Venta_hoy,Client_Sum_Venta_uni_hoy,Client_Sum_venta_div_venta_uni
0,26,8.606056e+04,4597.0,18.721027
1,60,1.424915e+06,30989.0,45.981328
2,65,7.988884e+05,49315.0,16.199705
3,101,1.420980e+04,616.0,23.067858
4,105,2.373410e+05,13616.0,17.431037


In [15]:
client_sum.describe()

,Cliente_ID,Client_Sum_Venta_hoy,Client_Sum_Venta_uni_hoy,Client_Sum_venta_div_venta_uni
count,8.619790e+05,8.619790e+05,8.619790e+05,861979.000000
mean,2.264008e+06,4.238666e+03,4.521941e+02,8.298836
std,2.886089e+06,1.201346e+05,1.436146e+04,6.029168
min,2.600000e+01,5.000000e-01,1.000000e+00,0.168948
25%,6.173875e+05,4.262200e+02,6.000000e+01,6.270000
50%,1.800685e+06,1.245290e+03,1.730000e+02,7.326471
75%,4.297210e+06,3.377105e+03,4.450000e+02,8.766549
max,2.015152e+09,1.096880e+08,1.325432e+07,647.360003


In [16]:
client_sum.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 861979 entries, 0 to 861978
Data columns (total 4 columns):
Cliente_ID                        861979 non-null int64
Client_Sum_Venta_hoy              861979 non-null float32
Client_Sum_Venta_uni_hoy          861979 non-null float64
Client_Sum_venta_div_venta_uni    861979 non-null float64
dtypes: float32(1), float64(2), int64(1)
memory usage: 29.6 MB


In [17]:
client_sum[['Cliente_ID']]=client_sum[['Cliente_ID']].astype('uint32')
client_sum[['Client_Sum_Venta_hoy']]=client_sum[['Client_Sum_Venta_hoy']].astype('float32')
client_sum[['Client_Sum_Venta_uni_hoy']]=client_sum[['Client_Sum_Venta_uni_hoy']].astype('float32')
client_sum[['Client_Sum_venta_div_venta_uni']]=client_sum[['Client_Sum_venta_div_venta_uni']].astype('float32')

In [18]:
client_sum.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 861979 entries, 0 to 861978
Data columns (total 4 columns):
Cliente_ID                        861979 non-null uint32
Client_Sum_Venta_hoy              861979 non-null float32
Client_Sum_Venta_uni_hoy          861979 non-null float32
Client_Sum_venta_div_venta_uni    861979 non-null float32
dtypes: float32(3), uint32(1)
memory usage: 19.7 MB


In [19]:
FE.train.head()

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0,Lag1,Lag2,Lag3,weightppieces
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,2,709,4,3.0,False,1.595831,0.0,0.0,0.0,60
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,2,712,4,4.0,False,1.177221,0.0,0.0,0.0,67
2,3,1110,7,3301,15766,1238,4,39.320000,0,0.0,...,2,630,4,4.0,False,1.177221,0.0,0.0,0.0,70
3,3,1110,7,3301,15766,1240,4,33.520000,0,0.0,...,4,480,4,4.0,False,1.962035,0.0,0.0,0.0,31
4,3,1110,7,3301,15766,1242,3,22.920000,0,0.0,...,6,271,4,3.0,False,1.271222,0.0,0.0,0.0,17


In [20]:
FE.train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53165116 entries, 0 to 53364882
Data columns (total 24 columns):
Semana                                          uint8
Agencia_ID                                      uint16
Canal_ID                                        uint8
Ruta_SAK                                        uint16
Cliente_ID                                      uint32
Producto_ID                                     uint16
Venta_uni_hoy                                   uint16
Venta_hoy                                       float32
Dev_uni_proxima                                 uint32
Dev_proxima                                     float32
Demanda_uni_equil                               float64
Town_ID                                         uint16
State_ID                                        uint8
weight                                          uint16
pieces                                          uint8
Prod_name_ID                                    uint16
Brand_

In [21]:
FE.train = pd.merge(FE.train, client_sum, on='Cliente_ID', how='left', sort=False, copy=False)
gc.collect()

639

In [22]:
prod_name_sum_venta_hoy = FE.train.groupby('Prod_name_ID',as_index=False)['Venta_hoy'].sum()
prod_name_sum_venta_uni_hoy = FE.train.groupby('Prod_name_ID',as_index=False)['Venta_uni_hoy'].sum()
prod_name_sum = pd.merge(prod_name_sum_venta_hoy, prod_name_sum_venta_uni_hoy, on='Prod_name_ID', how='left', sort=False,copy=False)
prod_name_sum.loc[:,"prod_name_sum_venta_div_venta_uni"] = (prod_name_sum["Venta_hoy"].values/prod_name_sum["Venta_uni_hoy"].values)
prod_name_sum.columns = ['Prod_name_ID', 'prod_name_sum_Venta_hoy', 'prod_name_sum_Venta_uni_hoy', 'prod_name_sum_venta_div_venta_uni']

In [23]:
prod_name_sum[['Prod_name_ID']]=prod_name_sum[['Prod_name_ID']].astype('uint16')
prod_name_sum[['prod_name_sum_Venta_hoy']]=prod_name_sum[['prod_name_sum_Venta_hoy']].astype('float32')
prod_name_sum[['prod_name_sum_Venta_uni_hoy']]=prod_name_sum[['prod_name_sum_Venta_uni_hoy']].astype('float32')
prod_name_sum[['prod_name_sum_venta_div_venta_uni']]=prod_name_sum[['prod_name_sum_venta_div_venta_uni']].astype('float32')

In [24]:
FE.train = pd.merge(FE.train, prod_name_sum, on='Prod_name_ID', how='left', sort=False, copy=False)
gc.collect()

77

In [25]:
FE.train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53165116 entries, 0 to 53165115
Data columns (total 30 columns):
Semana                                          uint8
Agencia_ID                                      uint16
Canal_ID                                        uint8
Ruta_SAK                                        uint16
Cliente_ID                                      uint32
Producto_ID                                     uint16
Venta_uni_hoy                                   uint16
Venta_hoy                                       float32
Dev_uni_proxima                                 uint32
Dev_proxima                                     float32
Demanda_uni_equil                               float64
Town_ID                                         uint16
State_ID                                        uint8
weight                                          uint16
pieces                                          uint8
Prod_name_ID                                    uint16
Brand_

In [26]:
Producto_sum_venta_hoy = FE.train.groupby('Producto_ID',as_index=False)['Venta_hoy'].sum()
Producto_sum_venta_uni_hoy = FE.train.groupby('Producto_ID',as_index=False)['Venta_uni_hoy'].sum()
Producto_sum = pd.merge(Producto_sum_venta_hoy, Producto_sum_venta_uni_hoy, on='Producto_ID', how='left', sort=False,copy=False)
Producto_sum.loc[:,"Producto_sum_venta_div_venta_uni"] = (Producto_sum["Venta_hoy"].values/Producto_sum["Venta_uni_hoy"].values)
Producto_sum.columns = ['Producto_ID', 'Producto_sum_Venta_hoy', 'Producto_sum_Venta_uni_hoy', 'Producto_sum_venta_div_venta_uni']
Producto_sum[['Producto_ID']]=Producto_sum[['Producto_ID']].astype('uint16')
Producto_sum[['Producto_sum_Venta_hoy']]=Producto_sum[['Producto_sum_Venta_hoy']].astype('float32')
Producto_sum[['Producto_sum_Venta_uni_hoy']]=Producto_sum[['Producto_sum_Venta_uni_hoy']].astype('float32')
Producto_sum[['Producto_sum_venta_div_venta_uni']]=Producto_sum[['Producto_sum_venta_div_venta_uni']].astype('float32')
FE.train = pd.merge(FE.train, Producto_sum, on='Producto_ID', how='left', sort=False, copy=False)
gc.collect()

70

In [27]:
FE.test1 = pd.merge(FE.test1, client_sum, on='Cliente_ID', how='left', sort=False, copy=False)
FE.test2 = pd.merge(FE.test2, client_sum, on='Cliente_ID', how='left', sort=False, copy=False)
FE.test1 = pd.merge(FE.test1, prod_name_sum, on='Prod_name_ID', how='left', sort=False, copy=False)
FE.test2 = pd.merge(FE.test2, prod_name_sum, on='Prod_name_ID', how='left', sort=False, copy=False)
FE.test1 = pd.merge(FE.test1, Producto_sum, on='Producto_ID', how='left', sort=False, copy=False)
FE.test2 = pd.merge(FE.test2, Producto_sum, on='Producto_ID', how='left', sort=False, copy=False)
FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)
gc.collect()

141

In [28]:
FE.SaveDataFrameToHdf('both')


In [29]:
FE.train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53165116 entries, 0 to 53165115
Data columns (total 33 columns):
Semana                                          uint8
Agencia_ID                                      uint16
Canal_ID                                        uint8
Ruta_SAK                                        uint16
Cliente_ID                                      uint32
Producto_ID                                     uint16
Venta_uni_hoy                                   uint16
Venta_hoy                                       float32
Dev_uni_proxima                                 uint32
Dev_proxima                                     float32
Demanda_uni_equil                               float64
Town_ID                                         uint16
State_ID                                        uint8
weight                                          uint16
pieces                                          uint8
Prod_name_ID                                    uint16
Brand_

In [30]:
Feature1 = FE.train[["Producto_ID","Demanda_uni_equil","Venta_uni_hoy"]].copy()
gc.collect()
display(Feature1.head(2))
print "Venta_Uni_Hoy = 0: " ,((Feature1.Venta_uni_hoy == 0).sum())
print "Shape of New Dataframe..: ", str((Feature1.shape[0]))
gc.collect()
Feature1 = Feature1[Feature1.Venta_uni_hoy != 0]
print "Shape of New Dataframe after deleting Venta_Uni_Hoy = 0..: ",(Feature1.shape[0])
gc.collect()
Feature1.loc[:,"Demanda_uni_equil"] = np.round( np.expm1(Feature1["Demanda_uni_equil"].values) )
display(Feature1[Feature1.Demanda_uni_equil.values != Feature1.Venta_uni_hoy.values].head(10))
Feature1 = Feature1.groupby("Producto_ID").sum()
gc.collect()
Feature1.loc[:,"Producto_ID_sum_demanda_divide_sum_venta_uni"] = Feature1.Demanda_uni_equil.values / Feature1.Venta_uni_hoy.values
Feature1 = pd.DataFrame(Feature1["Producto_ID_sum_demanda_divide_sum_venta_uni"])
gc.collect()
print Feature1.mean()
display(Feature1.head(10))
FE.train = FE.train.merge( Feature1, left_on="Producto_ID",
                    right_index=True, how='left', sort=False,copy=False)
FE.test1 = FE.test1.merge( Feature1, left_on="Producto_ID",
                    right_index=True, how='left', sort=False,copy=False)
FE.test2 = FE.test2.merge( Feature1, left_on="Producto_ID",
                    right_index=True, how='left', sort=False,copy=False)
display(FE.train.head(2))
display(FE.test1.head(2))
display(FE.test2.head(2))

,Producto_ID,Demanda_uni_equil,Venta_uni_hoy
0,1212,1.386294,3
1,1216,1.609438,4


Venta_Uni_Hoy = 0:  0
Shape of New Dataframe..:  53165116
Shape of New Dataframe after deleting Venta_Uni_Hoy = 0..:  53165116


,Producto_ID,Demanda_uni_equil,Venta_uni_hoy
116,47611,1.0,2
282,1309,1.0,2
287,30571,6.0,7
291,34053,0.0,3
381,35144,8.0,10
483,1309,6.0,7
591,30571,0.0,3
1044,31717,2.0,5
1258,1125,0.0,16
1271,1250,0.0,2


Producto_ID_sum_demanda_divide_sum_venta_uni    0.960515
dtype: float64


,Producto_ID_sum_demanda_divide_sum_venta_uni
Producto_ID,
41,0.999499
53,1.000000
72,0.990105
73,0.976146
100,0.949527
106,0.985925
107,1.000000
108,1.000000
123,0.995179


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Client_Sum_Venta_hoy,Client_Sum_Venta_uni_hoy,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,4351.620117,636.0,6.84217,50557572.0,4348170.0,11.627322,21736028.0,2596974.0,8.369752,0.982730
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,4351.620117,636.0,6.84217,42279768.0,4053394.0,10.430708,20126136.0,2396759.0,8.397230,0.985235


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Client_Sum_Venta_hoy,Client_Sum_Venta_uni_hoy,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni
0,8,1110,7,3301,15766,1212,4,33.520000,0,0.0,...,4351.620117,636.0,6.84217,50557572.0,4348170.0,11.627322,21736028.0,2596974.0,8.369752,0.982730
1,8,1110,7,3301,15766,1216,5,41.900002,0,0.0,...,4351.620117,636.0,6.84217,42279768.0,4053394.0,10.430708,20126136.0,2396759.0,8.397230,0.985235


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Client_Sum_Venta_hoy,Client_Sum_Venta_uni_hoy,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni
0,9,1110,7,3301,15766,1212,1,8.38,0,0.0,...,4351.620117,636.0,6.84217,50557572.0,4348170.0,11.627322,21736028.0,2596974.0,8.369752,0.982730
1,9,1110,7,3301,15766,1238,2,19.66,0,0.0,...,4351.620117,636.0,6.84217,32648700.0,3267844.0,9.990899,28693384.0,2881029.0,9.959422,0.984176


In [31]:
Feature1 = FE.train[["Prod_name_ID","Demanda_uni_equil","Venta_uni_hoy"]].copy()
gc.collect()
display(Feature1.head(2))
print "Venta_Uni_Hoy = 0: " ,((Feature1.Venta_uni_hoy == 0).sum())
print "Shape of New Dataframe..: ", str((Feature1.shape[0]))
gc.collect()
Feature1 = Feature1[Feature1.Venta_uni_hoy != 0]
print "Shape of New Dataframe after deleting Venta_Uni_Hoy = 0..: ",(Feature1.shape[0])
gc.collect()
Feature1.loc[:,"Demanda_uni_equil"] = np.round( np.expm1(Feature1["Demanda_uni_equil"].values) )
display(Feature1[Feature1.Demanda_uni_equil.values != Feature1.Venta_uni_hoy.values].head(10))
Feature1 = Feature1.groupby("Prod_name_ID").sum()
gc.collect()
Feature1.loc[:,"Prod_name_ID_sum_demanda_divide_sum_venta_uni"] = Feature1.Demanda_uni_equil.values / Feature1.Venta_uni_hoy.values
Feature1 = pd.DataFrame(Feature1["Prod_name_ID_sum_demanda_divide_sum_venta_uni"])
gc.collect()
print Feature1.mean()
display(Feature1.head(10))
FE.train = FE.train.merge( Feature1, left_on="Prod_name_ID",
                    right_index=True, how='left', sort=False,copy=False)
FE.test1 = FE.test1.merge( Feature1, left_on="Prod_name_ID",
                    right_index=True, how='left', sort=False,copy=False)
FE.test2 = FE.test2.merge( Feature1, left_on="Prod_name_ID",
                    right_index=True, how='left', sort=False,copy=False)
display(FE.train.head(2))
display(FE.test1.head(2))
display(FE.test2.head(2))

,Prod_name_ID,Demanda_uni_equil,Venta_uni_hoy
0,709,1.386294,3
1,712,1.609438,4


Venta_Uni_Hoy = 0:  0
Shape of New Dataframe..:  53165116
Shape of New Dataframe after deleting Venta_Uni_Hoy = 0..:  53165116


,Prod_name_ID,Demanda_uni_equil,Venta_uni_hoy
116,78,1.0,2
282,82,1.0,2
287,45,6.0,7
291,544,0.0,3
381,961,8.0,10
483,82,6.0,7
591,45,0.0,3
1044,71,2.0,5
1258,876,0.0,16
1271,266,0.0,2


Prod_name_ID_sum_demanda_divide_sum_venta_uni    0.959098
dtype: float64


,Prod_name_ID_sum_demanda_divide_sum_venta_uni
Prod_name_ID,
0,0.986357
1,0.938872
2,0.947832
3,0.973487
4,0.968796
5,0.980256
6,0.959691
9,0.981243
10,0.981995


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Client_Sum_Venta_uni_hoy,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,636.0,6.84217,50557572.0,4348170.0,11.627322,21736028.0,2596974.0,8.369752,0.982730,0.980954
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,636.0,6.84217,42279768.0,4053394.0,10.430708,20126136.0,2396759.0,8.397230,0.985235,0.983903


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Client_Sum_Venta_uni_hoy,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni
0,8,1110,7,3301,15766,1212,4,33.520000,0,0.0,...,636.0,6.84217,50557572.0,4348170.0,11.627322,21736028.0,2596974.0,8.369752,0.982730,0.980954
1,8,1110,7,3301,15766,1216,5,41.900002,0,0.0,...,636.0,6.84217,42279768.0,4053394.0,10.430708,20126136.0,2396759.0,8.397230,0.985235,0.983903


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Client_Sum_Venta_uni_hoy,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni
0,9,1110,7,3301,15766,1212,1,8.38,0,0.0,...,636.0,6.84217,50557572.0,4348170.0,11.627322,21736028.0,2596974.0,8.369752,0.982730,0.980954
1,9,1110,7,3301,15766,1238,2,19.66,0,0.0,...,636.0,6.84217,32648700.0,3267844.0,9.990899,28693384.0,2881029.0,9.959422,0.984176,0.984168


In [32]:
Feature1 = FE.train[["Cliente_ID","Demanda_uni_equil","Venta_uni_hoy"]].copy()
gc.collect()
display(Feature1.head(2))
print "Venta_Uni_Hoy = 0: " ,((Feature1.Venta_uni_hoy == 0).sum())
print "Shape of New Dataframe..: ", str((Feature1.shape[0]))
gc.collect()
Feature1 = Feature1[Feature1.Venta_uni_hoy != 0]
print "Shape of New Dataframe after deleting Venta_Uni_Hoy = 0..: ",(Feature1.shape[0])
gc.collect()
Feature1.loc[:,"Demanda_uni_equil"] = np.round( np.expm1(Feature1["Demanda_uni_equil"].values) )
display(Feature1[Feature1.Demanda_uni_equil.values != Feature1.Venta_uni_hoy.values].head(10))
Feature1 = Feature1.groupby("Cliente_ID").sum()
gc.collect()
Feature1.loc[:,"Cliente_ID_sum_demanda_divide_sum_venta_uni"] = Feature1.Demanda_uni_equil.values / Feature1.Venta_uni_hoy.values
Feature1 = pd.DataFrame(Feature1["Cliente_ID_sum_demanda_divide_sum_venta_uni"])
gc.collect()
print Feature1.mean()
display(Feature1.head(10))
FE.train = FE.train.merge( Feature1, left_on="Cliente_ID",
                    right_index=True, how='left', sort=False,copy=False)
FE.test1 = FE.test1.merge( Feature1, left_on="Cliente_ID",
                    right_index=True, how='left', sort=False,copy=False)
FE.test2 = FE.test2.merge( Feature1, left_on="Cliente_ID",
                    right_index=True, how='left', sort=False,copy=False)
display(FE.train.head(2))
display(FE.test1.head(2))
display(FE.test2.head(2))

,Cliente_ID,Demanda_uni_equil,Venta_uni_hoy
0,15766,1.386294,3
1,15766,1.609438,4


Venta_Uni_Hoy = 0:  0
Shape of New Dataframe..:  53165116
Shape of New Dataframe after deleting Venta_Uni_Hoy = 0..:  53165116


,Cliente_ID,Demanda_uni_equil,Venta_uni_hoy
116,319641,1.0,2
282,1603500,1.0,2
287,1603500,6.0,7
291,1603500,0.0,3
381,2331717,8.0,10
483,4532486,6.0,7
591,319684,0.0,3
1044,1690065,2.0,5
1258,384195,0.0,16
1271,817296,0.0,2


Cliente_ID_sum_demanda_divide_sum_venta_uni    0.984827
dtype: float64


,Cliente_ID_sum_demanda_divide_sum_venta_uni
Cliente_ID,
26,0.928649
60,1.000000
65,0.999838
101,1.000000
105,1.000000
106,1.000000
107,0.951031
215,1.000000
262,1.000000


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni,Cliente_ID_sum_demanda_divide_sum_venta_uni
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,6.84217,50557572.0,4348170.0,11.627322,21736028.0,2596974.0,8.369752,0.982730,0.980954,0.996855
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,6.84217,42279768.0,4053394.0,10.430708,20126136.0,2396759.0,8.397230,0.985235,0.983903,0.996855


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni,Cliente_ID_sum_demanda_divide_sum_venta_uni
0,8,1110,7,3301,15766,1212,4,33.520000,0,0.0,...,6.84217,50557572.0,4348170.0,11.627322,21736028.0,2596974.0,8.369752,0.982730,0.980954,0.996855
1,8,1110,7,3301,15766,1216,5,41.900002,0,0.0,...,6.84217,42279768.0,4053394.0,10.430708,20126136.0,2396759.0,8.397230,0.985235,0.983903,0.996855


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni,Cliente_ID_sum_demanda_divide_sum_venta_uni
0,9,1110,7,3301,15766,1212,1,8.38,0,0.0,...,6.84217,50557572.0,4348170.0,11.627322,21736028.0,2596974.0,8.369752,0.982730,0.980954,0.996855
1,9,1110,7,3301,15766,1238,2,19.66,0,0.0,...,6.84217,32648700.0,3267844.0,9.990899,28693384.0,2881029.0,9.959422,0.984176,0.984168,0.996855


In [ ]:
FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)
gc.collect()
FE.SaveDataFrameToHdf('both')

In [ ]:
clientPerTown = FE.train[["Town_ID","Cliente_ID","Lag0"]].copy()

clientPerTown = clientPerTown.groupby(["Town_ID","Cliente_ID"])

clientPerTown = clientPerTown.count()
gc.collect()

clientPerTown.reset_index(inplace=True)

clientPerTown = clientPerTown.groupby("Town_ID").count()

clientPerTown = pd.DataFrame( { "ClientPerTown" : clientPerTown.Cliente_ID} )

gc.collect()

FE.train = FE.train.merge( clientPerTown, left_on="Town_ID",
                    right_index=True, how='left', sort=False,copy=False)
FE.test1 = FE.test1.merge( clientPerTown, left_on="Town_ID",
                    right_index=True, how='left', sort=False,copy=False)
FE.test2 = FE.test2.merge( clientPerTown, left_on="Town_ID",
                    right_index=True, how='left', sort=False,copy=False)
display(FE.train.head(2))
display(FE.test1.head(2))
display(FE.test2.head(2))

FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)
gc.collect()


client_return_sum = FE.train.groupby('Cliente_ID',as_index=False)['DemandaNotEqualTheDifferenceOfVentaUniAndDev'].sum()
client_return_count = FE.train.groupby('Cliente_ID',as_index=False)['DemandaNotEqualTheDifferenceOfVentaUniAndDev'].count()
client_return = pd.merge(client_return_sum, client_return_count, on='Cliente_ID', how='left', sort=False,copy=False)

client_return.loc[:,"Client_return_rate"] = (client_return["DemandaNotEqualTheDifferenceOfVentaUniAndDev_x"].values/client_return["DemandaNotEqualTheDifferenceOfVentaUniAndDev_y"].values)
client_return.columns = ['Cliente_ID', 'client_return_sum', 'client_return_count', 'Client_return_rate']

FE.train = pd.merge(FE.train, client_return, on='Cliente_ID', how='left', sort=False, copy=False)
FE.test1 = pd.merge(FE.test1, client_return, on='Cliente_ID', how='left', sort=False, copy=False)
FE.test2 = pd.merge(FE.test2, client_return, on='Cliente_ID', how='left', sort=False, copy=False)

FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)
gc.collect()


producto_return_sum = FE.train.groupby('Producto_ID',as_index=False)['DemandaNotEqualTheDifferenceOfVentaUniAndDev'].sum()
producto_return_count = FE.train.groupby('Producto_ID',as_index=False)['DemandaNotEqualTheDifferenceOfVentaUniAndDev'].count()
producto_return = pd.merge(producto_return_sum, producto_return_count, on='Producto_ID', how='left', sort=False,copy=False)

producto_return.loc[:,"producto_return_rate"] = (producto_return["DemandaNotEqualTheDifferenceOfVentaUniAndDev_x"].values/producto_return["DemandaNotEqualTheDifferenceOfVentaUniAndDev_y"].values)
producto_return.columns = ['Producto_ID', 'producto_return_sum', 'producto_return_count', 'producto_return_rate']

FE.train = pd.merge(FE.train, producto_return, on='Producto_ID', how='left', sort=False, copy=False)
FE.test1 = pd.merge(FE.test1, producto_return, on='Producto_ID', how='left', sort=False, copy=False)
FE.test2 = pd.merge(FE.test2, producto_return, on='Producto_ID', how='left', sort=False, copy=False)

FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)
gc.collect()

prod_name_return_sum = FE.train.groupby('Prod_name_ID',as_index=False)['DemandaNotEqualTheDifferenceOfVentaUniAndDev'].sum()
prod_name_return_count = FE.train.groupby('Prod_name_ID',as_index=False)['DemandaNotEqualTheDifferenceOfVentaUniAndDev'].count()
producto_return = pd.merge(prod_name_return_sum, prod_name_return_count, on='Prod_name_ID', how='left', sort=False,copy=False)

producto_return.loc[:,"prod_name_return_rate"] = (producto_return["DemandaNotEqualTheDifferenceOfVentaUniAndDev_x"].values/producto_return["DemandaNotEqualTheDifferenceOfVentaUniAndDev_y"].values)
producto_return.columns = ['Prod_name_ID', 'prod_name_return_sum', 'prod_name_return_count', 'prod_name_return_rate']

FE.train = pd.merge(FE.train, producto_return, on='Prod_name_ID', how='left', sort=False, copy=False)
FE.test1 = pd.merge(FE.test1, producto_return, on='Prod_name_ID', how='left', sort=False, copy=False)
FE.test2 = pd.merge(FE.test2, producto_return, on='Prod_name_ID', how='left', sort=False, copy=False)

FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)
gc.collect()
FE.SaveDataFrameToHdf('both')
gc.collect()

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni,Cliente_ID_sum_demanda_divide_sum_venta_uni,ClientPerTown
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,50557572.0,4348170.0,11.627322,21736028.0,2596974.0,8.369752,0.982730,0.980954,0.996855,4204
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,42279768.0,4053394.0,10.430708,20126136.0,2396759.0,8.397230,0.985235,0.983903,0.996855,4204


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni,Cliente_ID_sum_demanda_divide_sum_venta_uni,ClientPerTown
0,8,1110,7,3301,15766,1212,4,33.520000,0,0.0,...,50557572.0,4348170.0,11.627322,21736028.0,2596974.0,8.369752,0.982730,0.980954,0.996855,4204
1,8,1110,7,3301,15766,1216,5,41.900002,0,0.0,...,42279768.0,4053394.0,10.430708,20126136.0,2396759.0,8.397230,0.985235,0.983903,0.996855,4204


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni,Cliente_ID_sum_demanda_divide_sum_venta_uni,ClientPerTown
0,9,1110,7,3301,15766,1212,1,8.38,0,0.0,...,50557572.0,4348170.0,11.627322,21736028.0,2596974.0,8.369752,0.982730,0.980954,0.996855,4204
1,9,1110,7,3301,15766,1238,2,19.66,0,0.0,...,32648700.0,3267844.0,9.990899,28693384.0,2881029.0,9.959422,0.984176,0.984168,0.996855,4204
